In [0]:
#!import "../DataModel/DataStructure"

In [0]:
public record CashflowIdentity{
    public string DataNode {get; set;}
    public string AmountType {get; set;}
    public int? AccidentYear {get; set;}
}

# Storage

In [0]:
public class CashflowStorage
{   
    private readonly IDataSource querySource; 
    private readonly IWorkspace workspace;
    private readonly Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache;
    private readonly ImportArgs args;

    //Dimensions
    public HashSet<string> EarningsAmountTypes  { get; private set; }
    public HashSet<string> SolvencyBsAmountTypes  { get; private set; }
    public HashSet<string> IfrsBsAmountTypes  { get; private set; }

    public CashflowDefinition[] CashflowDefinitionsByIdentity { get; private set; }
    public BasicCashflowDefinition[] BasicCashflowDefinitionsByIdentity { get; private set; }
    public BoundedCashflowDefinition[] BoundedCashflowDefinitionsByIdentity { get; private set; }
    public ReferencedCashflowDefinition[] ReferencedCashflowDefinitionsByIdentity { get; private set; }

       
    public CashflowIdentity[] Identities {get; set;}

    // Constructor
    public CashflowStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
    {
        this.querySource = querySource;
        this.workspace = workspace;
        hierarchyCache = workspace.ToHierarchicalDimensionCache();
        this.args = args;
    }
    
    // Initialize
    public async Task InitializeAsync()
    {   
        // Dimensions
        EarningsAmountTypes = (await workspace.Query<EarningsAmountType>().ToArrayAsync()).Select(x => x.SystemName).ToHashSet();
        SolvencyBsAmountTypes = (await workspace.Query<SolvencyBsAmountType>().ToArrayAsync()).Select(x => x.SystemName).ToHashSet();
        IfrsBsAmountTypes = (await workspace.Query<IfrsBsAmountType>().ToArrayAsync()).Select(x => x.SystemName).ToHashSet();
        
        // Identities
        CashflowDefinitionsByIdentity = await workspace.Query<CashflowDefinition>().ToArrayAsync();
        BasicCashflowDefinitionsByIdentity = await workspace.Query<BasicCashflowDefinition>().ToArrayAsync();
        BoundedCashflowDefinitionsByIdentity = await workspace.Query<BoundedCashflowDefinition>().ToArrayAsync();
        ReferencedCashflowDefinitionsByIdentity = await workspace.Query<ReferencedCashflowDefinition>().ToArrayAsync();

        Identities = CashflowDefinitionsByIdentity.Select(x => new CashflowIdentity() 
            { 
                DataNode = x.DataNode,
                AmountType = x.AmountType,
                AccidentYear = x.AccidentYear
            }
        ).Distinct().ToArray();

    }

   
   // Getters
    public IEnumerable<BasicCashflowDefinition> GetBasicCashflowDefinitions(CashflowIdentity identity) => 
        BasicCashflowDefinitionsByIdentity.Where(x => x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);


    public IEnumerable<BoundedCashflowDefinition> GetBoundedCashflowDefinitions(CashflowIdentity identity) => 
        BoundedCashflowDefinitionsByIdentity.Where(x => x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);
    
    public IEnumerable<ReferencedCashflowDefinition> GetReferencedCashflowDefinition(CashflowIdentity identity) => 
        ReferencedCashflowDefinitionsByIdentity.Where(x => x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);

}